In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

## Reading the data 

In [141]:
PROJECT_DIR = f"E:\ML\Ml-porject\Supervised ml project\Flight_price_predication"
DATA_DIR = "data"

In [142]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR,file_name)
    return pd.read_csv(file_path)

In [143]:
df = get_data("flight")

In [144]:
df.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


## preliminary observation

In [146]:
df.duplicated().sum()

np.int64(220)

In [147]:
(df.loc[df.duplicated(keep=False)]
       .sort_values(["Airline","Date_of_Journey","Source","Destination"]))

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### Problem we need to solve to clean the data 
 Remvoe the missing values in route and Total_Stops Column\
 Fixing the datatypes of  different columns [Date_of_Journey,Dep_Time,Arrival_Time]\
 Duration column ko clearn kr na hai and datatypes change kr na hai\
 Total_Stops to be comverted into interger\
 There are 220 duplicate row in our dataset

## Detailed Analysis

### Airline

In [148]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [149]:
(
df
.Airline
.str.replace("Premium economy","")
.str.replace("Business","")
.str.title()
.unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

### Date of journay

In [150]:
df.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [151]:
pd.to_datetime(df.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [152]:
df['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [153]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


## destination

In [154]:
df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_Time

In [155]:
df.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [156]:
df.Dep_Time.loc[lambda ser: ser.str.contains("[^0-9:]")]  #0-9 this means value from 0-9 and : use for separaerd like above this ^ is means not like this show that/

Series([], Name: Dep_Time, dtype: object)

In [157]:
pd.to_datetime(df.Dep_Time).dt.time

C:\Users\sankb\AppData\Local\Temp\ipykernel_7824\3599857698.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

## arrival_time 

In [158]:
df['Arrival_Time']

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [159]:
(
    df
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)
    .str.get(1)
)

0        22 Mar
2        10 Jun
6        13 Mar
7        02 Mar
8        13 Mar
          ...  
10666    13 Jun
10667    13 Mar
10672    28 Jun
10673    28 May
10674    13 Mar
Name: Arrival_Time, Length: 4335, dtype: object

## duration

In [160]:
df['Duration']

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [161]:
(
    df
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [162]:
(
    df
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    
)

6474    5m
Name: Duration, dtype: object

In [169]:
df.loc[6474]

Airline                        Air India
Date_of_Journey                6/03/2019
Source                            Mumbai
Destination                    Hyderabad
Route              BOM → GOI → PNQ → HYD
Dep_Time                           16:50
Arrival_Time                       16:55
Duration                              5m
Total_Stops                      2 stops
Additional_Info                  No info
Price                              17327
Name: 6474, dtype: object

In [199]:
(
	df
	.Duration
	.drop(index=6474)
	.str.split(" ", expand=True)
	.set_axis(["hour", "minute"], axis=1)
	.assign(
		hour=lambda df_: (
			df_
			.hour
			.str.replace("h", "")
			.astype(int)
			.mul(60)
		),
		minute=lambda df_: (
			df_
			.minute
			.str.replace("m", "")
			.fillna("0")
			.astype(int)
		)
	)
	.sum(axis=1)
	.rename("duration_minutes")
	.to_frame()
	.join(df.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


## Total Stops 

In [209]:
df.Total_Stops.unique()



array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [210]:
(
	df
	.Total_Stops
	.replace("non-stop", "0")
	.str.replace(" stops?", "", regex=True) # " stop" " stops"
	.pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

## Aditional_info

In [212]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

- The obersvastion indexed 6474 has duration of 5 min.this clearly wrong .we will delete this observation

##  Clearning Operations

In [202]:
def convert_to_minutes(ser):
	return (
		ser
		.str.split(" ", expand=True)
		.set_axis(["hour", "minute"], axis=1)
		.assign(
			hour=lambda df_: (
				df_
				.hour
				.str.replace("h", "")
				.astype(int)
				.mul(60)
			),
			minute=lambda df_: (
				df_
				.minute
				.str.replace("m", "")
				.fillna("0")
				.astype(int)
			)
		)
		.sum(axis=1)
	)

In [207]:
def clean_data(df):
	return (
		df
		.drop(index=[6474])
		.drop_duplicates()
		.assign(**{
			col: df[col].str.strip()
			for col in df.select_dtypes(include="O").columns
		})
		.rename(columns=str.lower)
		.assign(
			airline=lambda df_: (
				df_
				.airline
				.str.replace(" Premium economy", "")
				.str.replace(" Business", "")
				.str.title()
			),
			date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
			dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
			arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
			duration=lambda df_: df_.duration.pipe(convert_to_minutes),
			total_stops=lambda df_: (
				df_
				.total_stops
				.replace("non-stop", "0")
				.str.replace(" stops?", "", regex=True)
				.pipe(lambda ser: pd.to_numeric(ser))
			),
			additional_info=lambda df_: df_.additional_info.replace("No info", "No Info")
		)
		.drop(columns="route")
	)

In [208]:
df_cleand = clean_data(df)
df_cleand

C:\Users\sankb\AppData\Local\Temp\ipykernel_7824\3708629844.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
C:\Users\sankb\AppData\Local\Temp\ipykernel_7824\3708629844.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


## Slipting the data

In [213]:
flight_final = df_cleand.sample(1000)

In [214]:
X = flight_final.drop(columns="price")
y = flight_final.price.copy()

In [215]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


In [216]:
def export_data(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

	X.join(y).to_csv(file_path, index=False)

	return pd.read_csv(file_path).head()

In [217]:
export_data(X_train, y_train, "train")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-04-01,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No Info,3971
1,Jet Airways,2019-03-24,Kolkata,Banglore,21:10:00,08:10:00,660,1.0,In-flight meal not included,11134
2,Indigo,2019-03-09,Banglore,New Delhi,06:00:00,08:45:00,165,0.0,No Info,7648
3,Jet Airways,2019-03-09,Banglore,New Delhi,09:45:00,00:30:00,885,1.0,In-flight meal not included,10562
4,Jet Airways,2019-03-01,Banglore,New Delhi,05:45:00,11:25:00,340,1.0,Business class,79512


In [218]:
export_data(X_val, y_val, "val")


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-03-06,Delhi,Cochin,14:35:00,22:20:00,465,1.0,In-flight meal not included,7563
1,Jet Airways,2019-03-21,Delhi,Cochin,19:45:00,04:25:00,520,1.0,No Info,12557
2,Air India,2019-06-12,Kolkata,Banglore,05:50:00,23:15:00,1045,2.0,No Info,12723
3,Multiple Carriers,2019-06-15,Delhi,Cochin,06:40:00,19:15:00,755,1.0,No Info,8614
4,Indigo,2019-06-27,Banglore,Delhi,23:30:00,02:20:00,170,0.0,No Info,3943


In [ ]:
export_data(X_test, y_test, "test")
